# Thinking 1 : CNN 的参数共享指的是什么？

答：  
CNN 的主要特点是：先进行特征提取（feature extraction）得到 Feature map 然后放入全连接层进行学习。  
CNN 的参数共享指的是使用卷积核在图像的不同位置进行特征提取时，卷积核（权重）是不变的，以此使用不同的卷积核得到图像不同维度的 Feature Map。

# Thinking 2 ：为什么会用到batch normalization?

答：  
Batch-Normalization：在每一批的数据进行卷积前，都进行一次标准化处理。 （由于数据量大，无法全部放进去，因此进行分批处理） 
1. 由于卷积神经网络层级比较大，出现一定的偏差就会造成深度学习时每一层都会存在偏差，造成深度学习的效率变慢，因此为了提高学习的效率，在每一次卷积前都做一次标准化。
2. BN 通过标准化将数据规范到 N（0，1）的正态分布水平，使得非线性函数的输入值在一个敏感范围内, 缓解梯度消失的问题。

# Thinking 3 ：使用dropout可以解决什么问题？

答：  
关闭某些神经元，防止过拟合问题。

# Action 1 ： 使用任何神经网络框架，对CIFAR-10进行分类
>http://www.cs.toronto.edu/~kriz/cifar.html  
训练集 50000，测试集 10000  
图像大小 32*32 彩色  
10个分类：ariplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck  

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
#数据加载
train_data = datasets.CIFAR10(root = 'C:/nas/cifar10/', train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.CIFAR10(root = 'C:/nas/cifar10/', train = False, transform = transforms.ToTensor(), download = True)
#定义超参数
EPOCH = 60
BATCH_SIZE = 256
LR = 0.001

# 使用dataloader 进行分批
train_loader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE)

#使用 denseNet
model = torchvision.models.densenet121(pretrained = True)

# 预训练会下载一个文件

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import time
# 定义损失函数
criterion = nn.CrossEntropyLoss()

#定义优化器
optimizer = optim.Adam(model.parameters(), lr = LR)

#定义device
device = torch.device('cuda:0' if torch.cuda.is_available() else'cpu')
model.to(device)

# 训练
for epoch in range(EPOCH):
    start_time = time.time()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #前向传播
        outputs = model(inputs)
        # 计算损失函数
        loss = criterion(outputs, labels)
        # 清空梯度
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        #参数更新
        optimizer.step()
    print('epoch{} loss{} time{}'.format(epoch + 1, loss.item(), time.time() - start_time))

epoch1 loss0.5672690272331238 time40.95031452178955
epoch2 loss0.5810059905052185 time40.27595400810242
epoch3 loss0.34251824021339417 time40.63038897514343
epoch4 loss0.27401426434516907 time41.32640528678894
epoch5 loss0.26239970326423645 time40.442259550094604
epoch6 loss0.24376359581947327 time40.674166679382324
epoch7 loss0.3113354742527008 time40.59865212440491
epoch8 loss0.33181753754615784 time40.707913637161255
epoch9 loss0.12041361629962921 time40.71990156173706
epoch10 loss0.09377484768629074 time41.26325750350952
epoch11 loss0.07072024047374725 time40.723472356796265
epoch12 loss0.14862391352653503 time40.74365782737732
epoch13 loss0.18139901757240295 time40.75734996795654
epoch14 loss0.21069887280464172 time40.99097990989685
epoch15 loss0.13860833644866943 time44.604464292526245
epoch16 loss0.12817427515983582 time43.88355207443237
epoch17 loss0.12013576924800873 time43.3668212890625
epoch18 loss0.038227397948503494 time41.96275305747986
epoch19 loss0.1915787160396576 time

In [3]:
# 保存训练的模型
file_name = 'cifar10_densenet121epoch60lr0.001batch256.pt'
torch.save(model, file_name)
print(file_name+' saved')

cifar10_densenet121epoch60lr0.001batch256.pt saved


In [4]:
#测试
model = torch.load(file_name)
model.eval()
total = 0
correct = 0
for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    #前向传播
    out = model(images)
    #越策结果
    _, predicted = torch.max(out.data, 1)
    # 判断预测结果与实际结果是否一致
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

#输出识别率
print('10000张图像的准确率：{}'.format(100.0 * correct / total))

10000张图像的准确率：86.41


In [5]:
file_name = 'cifar10_densenet121epoch60lr0.001batch256.pt'
model = torch.load(file_name)

EPOCH = 40
BATCH_SIZE = 256
LR = 0.001

import time
# 定义损失函数
criterion = nn.CrossEntropyLoss()

#定义优化器
optimizer = optim.Adam(model.parameters(), lr = LR)

#定义device
device = torch.device('cuda:0' if torch.cuda.is_available() else'cpu')
model.to(device)

# 训练
for epoch in range(EPOCH):
    start_time = time.time()
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #前向传播
        outputs = model(inputs)
        # 计算损失函数
        loss = criterion(outputs, labels)
        # 清空梯度
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        #参数更新
        optimizer.step()
    print('epoch{} loss{} time{}'.format(epoch + 1, loss.item(), time.time() - start_time))

epoch1 loss0.05031668022274971 time42.50939679145813
epoch2 loss0.0178708303719759 time42.61570858955383
epoch3 loss0.014036168344318867 time42.92334794998169
epoch4 loss0.01648365892469883 time42.898672342300415
epoch5 loss0.01879817806184292 time42.734074115753174
epoch6 loss0.0004975419724360108 time42.67076230049133
epoch7 loss0.0010417394805699587 time42.75729203224182
epoch8 loss0.2045796811580658 time42.63768911361694
epoch9 loss0.058820534497499466 time42.666391372680664
epoch10 loss0.061279989778995514 time43.433658838272095
epoch11 loss0.07710004597902298 time43.35028386116028
epoch12 loss0.008122741244733334 time44.06403660774231
epoch13 loss0.043019261211156845 time45.71465611457825
epoch14 loss0.03414652496576309 time57.12015986442566
epoch15 loss0.059438031166791916 time64.43510603904724
epoch16 loss0.034043531864881516 time60.98513436317444
epoch17 loss0.02369821071624756 time60.59696173667908
epoch18 loss0.002959128934890032 time60.161499977111816
epoch19 loss0.08731635

In [6]:
# 保存训练的模型
file_name = 'cifar10_densenet121epoch100lr0.001batch256.pt'
torch.save(model, file_name)
print(file_name+' saved')

cifar10_densenet121epoch100lr0.001batch256.pt saved


In [7]:
#测试
model = torch.load(file_name)
model.eval()
total = 0
correct = 0
for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    #前向传播
    out = model(images)
    #越策结果
    _, predicted = torch.max(out.data, 1)
    # 判断预测结果与实际结果是否一致
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

#输出识别率
print('10000张图像的准确率：{}'.format(100.0 * correct / total))

10000张图像的准确率：85.63


## CUDA out memery:
```python
#查看进程
nvidia-smi
# 关闭进程
taskkill -PID 进程号 -F 
```
## 参考：DenseNet 源码构造方式https://www.pianshen.com/article/5919349111/  
## 原论文：https://arxiv.org/pdf/1707.06990.pdf&gt;%60_

# 本章任务

In [2]:
import xlrd
data = xlrd.open_workbook('L7-2自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-7):
    print(table.row(row_index)[:2])
for row_index in range(nrows-6, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

17
[text:'原理', text:'传统图像识别模型']
[text:'原理', text:'CNN的由来']
[text:'原理', text:'局部感受野，共享权重，池化']
[text:'原理', text:'平移不变与空间不变特性']
[text:'原理', text:'卷积核的作用']
[text:'原理', text:'Batch Normalization']
[text:'原理', text:'Pooling与Dropout']
[text:'工具', text:'AlexNet原理及使用']
[text:'原理', text:'DenseNet原理'] empty:''
[text:'课下思考与练习', empty:''] empty:''
[text:'Thinking2', text:'参数共享指的是什么？'] text:'能简要说明他们的作用（10points）\n'
[text:'Thinking3', text:'为什么会用到batch normalization ?'] text:'能简要说明BN的作用（10points）'
[text:'Thinking4', text:'使用dropout可以解决什么问题？'] text:'能简要解决的问题（10points）'
[text:'Action1', text:'使用任何神经网络框架，对CIFAR-10进行分类\nhttp://www.cs.toronto.edu/~kriz/cifar.html\n训练集 50000，测试集 10000\n图像大小 32*32 彩色\n10个分类：ariplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck'] text:'1、完成代码（30points）\n2、使用ResNet, DenseNet或其他网络（20points）\n3、Accuracy >90% （20points）'
